# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv
import pyproj
import json

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

In [ ]:

def kelvin_to_fahrenheit(temp):
    return (temp - 273.15) * 9/5 + 32

# Convert the 'Max Temp' column from Kelvin to Fahrenheit
city_data_df['Max Temp'] = city_data_df['Max Temp'].apply(kelvin_to_fahrenheit)
city_data_df 


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:

coordinates ={
    'latitude':city_data_df['Lat'],
    'longitude':city_data_df['Lng'],
    'city':city_data_df['City'],
    'humidity':city_data_df['Humidity']
}

coordinates_df = pd.DataFrame(coordinates)


# Configure the map plot
# YOUR CODE HERE
map_plot = coordinates_df.hvplot.points(
    'longitude',
    'latitude',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'humidity',
    scale = 1.5)
# Display the map
map_plot

#coordinates_df

In [ ]:
#humidity_type = coordinates_df['humidity'].dtypes
#print(humidity_type)
#clean_check_df = coordinates_df.dropna()
#clean_check_df

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
conditions_df = city_data_df[(city_data_df["Cloudiness"]<40) & (city_data_df["Humidity"]<50) &
                                (city_data_df["Max Temp"]>60) & (city_data_df["Max Temp"]<80)]


# Drop any rows with null values
conditions_df = conditions_df.dropna()
conditions_df
# Display sample data
conditions_df

### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
conditions_copy = conditions_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = conditions_copy.loc[:,['City','Country','Lat','Lng','Humidity']]
hotel_df['Hotel'] = ''

# Display sample data
hotel_df.head()

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'radius':radius,
    'categories':'accommodation.hotel',
    'apiKey':geoapify_key,
    #'key':"43172dd314e24eb0a69e400f78a709a5"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    name_address = response.json()
    
    # Convert the API response to JSON format
    name_address = response.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

name_address

In [ ]:
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_plot